In [ ]:
import json
import pandas as pd
import numpy as np
import psycopg2
import requests
import os

# Database 1 credentials
host = os.environ.get('HOST')
dbname = os.environ.get('DBNAME')
user = os.environ.get('USER')
password = os.environ.get('PASSWORD')

# Database 2 credentials
host2 = os.environ.get('HOST2')
dbname2 = os.environ.get('DBNAME2')
user2 = os.environ.get('USER2')
password2 = os.environ.get('PASSWORD2')

def insert_data(conn, df):
    try:
        cur = conn.cursor()
        # Create table in database
        cur.execute("CREATE TABLE IF NOT EXISTS weather(timestamp TIMESTAMP, precipitation FLOAT, relativehumidityhc FLOAT, solarradiation FLOAT, airtemperaturehc FLOAT, windspeedultrasonic FLOAT, winddirultrasonic FLOAT);")
        for index, row in df.iterrows():
            cur.execute(
                "INSERT INTO weather (timestamp, precipitation, relativehumidityhc, solarradiation, airtemperaturehc, windspeedultrasonic, winddirultrasonic) \
                VALUES (%s, %s, %s, %s, %s, %s, %s)",
                (
                    row['timestamp'],
                    row['precipitation'],
                    row['relativehumidityhc'],
                    row['solarradiation'],
                    row['airtemperaturehc'],
                    row['windspeedultrasonic'],
                    row['winddirultrasonic']
                )
            )
    except psycopg2.Error as e:
        print("Error: Inserting Rows")
        print(e)
        raise
    finally:
        cur.close()

def lambda_handler(event, context):
    # connect to an API
    url = 'https://data.bs.ch/api/explore/v2.1/catalog/datasets/100294/records?order_by=timestamp%20DESC&limit=1'
    response = requests.get(url)
    data = response.json()
    
    # Check if data is not empty
    if not data:
        print("No data received from the API.")
        return

    # Transform it
    df = pd.DataFrame(data['results'])
    
    # Load into databases
    try: 
        conn_db1 = psycopg2.connect(f"host={host} dbname={dbname} user={user} password={password}")
        conn_db2 = psycopg2.connect(f"host={host2} dbname={dbname2} user={user2} password={password2}")
        conn_db1.set_session(autocommit=True)
        conn_db2.set_session(autocommit=True)   
    
    except psycopg2.Error as e: 
        print("Error: Could not make connection to the Postgres databases")
        print(e)
        return
    
    try:
        insert_data(conn_db1, df)
        insert_data(conn_db2, df)
    except Exception as e:
        print("Error: Inserting data into databases")
        print(e)
    finally:
        conn_db1.close()
        conn_db2.close()
    